## Quickstart 

To forecast using inait models you need to specifiy the dataset, how many steps in the future you want to forecast (the so-called `forecasting_horizon`), and how many steps in the past you want to use to train the model (`observation_length`). 

### Prerequisites

In [ ]:
# Add parent directory to Python path for client imports
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd())))
import pandas as pd 

import plotly.express as px
pd.options.plotting.backend = "plotly"


from client.prediction_script import (
    create_payload_from_file,
    get_dataframe_from_response,
)
from client.utils import make_request
# 🔧 API Configuration
base_url = "https://"       # Add the URL of your inait forecast deployment/SaaS
auth_key = "your_key_here"  # Replace with your actual API key


# 🛡️ Validate authentication setup
if not auth_key:
    raise ValueError("❌ Authentication key is required. Please check your API credentials.")

if not base_url:
    raise ValueError("❌ Base URL is required. Please check your API endpoint.")

### Run your first prediction model

We will present an example using the classic Box & Jenkins dataset that contains the monthly number of airline passengers between 1949 and 1960 (source: Box, George EP, et al. Time series analysis: forecasting and control. John Wiley & Sons, 2015.). The goal is to predict the passengers for the next year, i.e., 12 steps ahead. 

In [ ]:
### Load  the data 
data_path = "../data/Airline.csv"
data = pd.read_csv(data_path, index_col=0)

# Configure prediction parameters
target_columns = "number_of_passengers" 
forecasting_horizon = 12  # Predict 12 time steps ahead
observation_length = 24   # Use last 24 points as historical context

# Create the prediction payload
# This packages your data and parameters into the format expected by the API
payload = create_payload_from_file(
    file_path=data_path,
    target_columns=target_columns,
    forecasting_horizon=forecasting_horizon,
    observation_length=observation_length,
)

# Send prediction request to the API
response = make_request(base_url + "/prediction", payload, auth_key=auth_key)

# Process the response and extract results
df, session_id = get_dataframe_from_response(response)

In [ ]:
from client.plot_script import create_plot_payload, plot_image, bytes_from_base64

# Create a plotting payload using our session from the prediction
# The session_id links this plot to our previous prediction request
payload = create_plot_payload(
    session_id, 
    plot_type = "prediction", 
    cutoff_date=None,      # No specific cutoff needed for prediction plots
    prediction_for=None    # Plot all predictions
)

# Request the plot from the API
response = make_request(base_url + "/plot", payload, auth_key=auth_key)

# Extract and display the generated image
# The API returns plots as base64-encoded images
image_bytes = bytes_from_base64(response["response"]["data"])
plot_image(image_bytes)